# ___

# [ Image and Laboratory Spectroscopy  ]

**Department of Applied Geoinformatics and Carthography, Charles University** 

*Lukas Brodsky lukas.brodsky@natur.cuni.cz*

    
___

# Soil VNIR spectroscopy with PLSR 

In this notebook, we are going to build a regression model using the PLSR in Python for Soil Organic Carbon prediction from spectroscopy data. 

It covers: 

1. Introduction to the difference PLS-regression; 
2. Present the basic code for PLSR; 
3. Discuss the data we want to analyze (Soil VNIR spectra to predict Soil Organic Carbon content); 
4. Build the model of using a cross-validation approach. 

## PLSR
PLSR, acronym of Partial Least Squares, is a widespread regression technique used to analyze near-infrared spectroscopy data to predict continuous variable. Another, linked technique, is PCR (Principal COmponent Regression), which is simply a regression model built using a number of principal components derived using Principal Component Analysis (PCA). Yet the PCR is simple, however it does not take into account anything other than the components. It does not take into account our labels (reference) y. It solely depends on the variation in the input independent variables (X). That is obviously not optimal, and PLSR is a way to improve. 


# Difference between PCR and PLSR

Both PLS and PCR perform multiple linear regression, that is they build a linear model, y = Xb + e; 
where X are the predictor variables and y is the response variable. 
In VNIR analysis, the X are spectral values (e.g. covering the range of 350 to 2500 nm), y is the quantity - or quantities - we want to calibrate for (in our case soil / geology continuous property of interest). Finally e is an error (nois in the data).

The matrix X of the spectral values contains highly correlated data. The correlation is unrelated to our soil / geology property of interest, and it may obscure the variations we want to measure. Both PCR and PLSR will remove  the correlation.

In the PCR, the set of measuremd X is transformed into equivalent $X'=XW$ by a linear transformation $W$, such that all the new 'spectra' (which are the principal components) are linear independent. In statistics $X'$ is called the **factor scores**.

The linear transformation in PCR is such that it minimises the covariance between the diffrent rows of $X'$. That means this process only uses the spectral data, not the response values.

In the PLSR instead of finding hyperplanes of maximum variance between the response and independent variables, it finds a linear regression model by projecting the predicted variables and the observable variables to a new space. A PLS model will try to find the multidimensional direction in the X space that explains the maximum multidimensional variance direction in the y space. PLS regression is particularly suited when the matrix of predictors has more variables than observations, and when there is multicollinearity among X values. By contrast, standard regression will fail in these cases (unless it is regularized). 

PLSR is based on finding a similar linear transformation, but accomplishes the same task by maximising the covariance between $Y$ and $X'$. In other words, PLSR takes into account both spectra and response values and in doing so will improve on some of the limitation on PCR. For these reasons PLSR is typically prefered analytical tool in the soil and geology spectroscopy. 

# PLSR in Python Scikit-learn
`sklearn` already has got a PLSR package. 
**API:**

* `from sklearn.cross_decomposition import PLSRegression` 

* `PLSRegression(n_components=2, *, scale=True, max_iter=500, tol=1e-06, copy=True)`

where 

   **n_components** (int) is the numbner of components to keep for the model (user devined, however, default=2)
    It should be in [1, min(n_samples, n_features, n_targets)].

   **scale** (bool) whether to scale X and Y (default=True). 

   **max_iter** (int) is the maximum number of iterations (default=500). 

   **tol** (float), tolerance used as convergence criteria (default=1e-06). The algorithm stops whenever the squared norm of u_i - u_{i-1} is less than tol, where u corresponds to the left singular vector.

   **copy** (bool)  whether to copy X and Y in fit before applying centering, and potentially scaling. If False, these operations will be done inplace, modifying both arrays. (default=True) 



**Basic PLSR practice**

1/ first run the model for several number of components and define the `n_components` we want to keep in our PLS-regression; 


2/ once the PLS object is defined, we fit the regression to the data `x` (the preditor) and `y` (the known response). 

3/ use the model to run a cross-validation experiment (e.g. 10 fold).

In [ ]:
# import the libraries
import os 
from sys import stdout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

### Open Soil Spectral Library 
Soil spectroscopy is the measurement of light absorption when light in the visible, near infrared or mid infrared (Vis–NIR–MIR) regions of the electromagnetic spectrum is applied to a soil surface. The proportion of the incident radiation reflected by soil is sensed through Vis–NIR–MIR reflectance spectroscopy. These characteristic spectra (see Fig. below) can then be used to estimate numerous soil attributes including: minerals, organic compounds and water.

Read more on the soil spectroscopy web page: https://soilspectroscopy.org 

Here we use a small subset of the soil spectra collected over the area of the Czech Republic for demonstration purposes coming from Eurostat LUCAS data set. 

### Read and explore the data 

In [ ]:
data_path = './'
fn = 'ossl_vnir_soil_organic_carbon_subset.csv'
data = pd.read_csv(os.path.join(data_path, fn))

In [ ]:
# the first column is original dat id 
data.rename(columns={list(data)[0]:'id'}, inplace=True)

In [ ]:
# check the content
data.head()

In [ ]:
# Which column strats the spectral data? 

In [ ]:
data.iloc[:, 3:].head()

In [ ]:
# Than define your arrayx of X and y for the PLSR modelling

In [ ]:
y = data['oc_usda.calc_wpct'].values
X = data.values[:, 3:]

In [ ]:
# What is the SOC data distribution?

In [ ]:
pd.DataFrame(y).hist()

In [ ]:
# Howabout histogram asymetry? 
# Can we improve? 

In [ ]:
y[1:5]

In [ ]:
# simple square root 
# if used, the data needs treatment after the model prediction to get the same scale for the SOC 
y_sqrt = np.sqrt(y)
pd.DataFrame(y_sqrt).hist()

#### Plot soil spectra

In [ ]:
# DataFrame stors the spectral values as separate columns. It starts at 452 nm and ends with 2500 nm,  
# while the OSSL uses step 2 nm

In [ ]:
# prepare the wavelength of the spectral records
wl = np.arange(452, 2501, 2)

In [ ]:
# Select one spectra and plot the soil spectral curve
sel_spectra = 100

In [ ]:
with plt.style.context('ggplot'):
    plt.plot(wl, X[1:5, :].T)
    plt.xlabel("Wavelengths (nm)")
    plt.ylabel("Reflectance (%)")

In [ ]:
# If required, data can be easily sorted by PCA and corrected with multiplicative scatter correction; 
# Another yet simple way to enhance the spectra is to perform second derivative; 

In [ ]:
# first smooth the spectra with Savitzky–Golay filter;  
# to increase the precision of the data without distorting the signal tendency
# by fitting successive sub-sets of adjacent data points with a low-degree polynomial by OLS; 
X_sg = savgol_filter(X, window_length = 17, polyorder=2)

In [ ]:
# and calculate the second derivative 
X_d2 = savgol_filter(X_sg, window_length = 17, polyorder=2, deriv=2) 

In [ ]:
# plot and see
plt.figure(figsize=(8, 4.5))
with plt.style.context('ggplot'):
    plt.plot(wl, X_d2[1, :].T)
    plt.xlabel("Wavelengths (nm)")
    plt.ylabel("Reflectance 2nd_drv")
    plt.show()

### Apply PLSR model to the data

In [ ]:
# function to find optimal number of components 

number_cv = 10

def optimise_pls_cv(X, y, n_comp):
    # PLSR object
    plsr = PLSRegression(n_components=n_comp)

    # cross-validation
    y_cv = cross_val_predict(plsr, X, y, cv=number_cv)

    # calculate scores
    r2 = r2_score(y, y_cv)
    mse = mean_squared_error(y, y_cv)
    rpd = y.std()/np.sqrt(mse)
    
    return (y_cv, r2, mse, rpd)

**Model quality evaluation** 

R2 ... coefficient of determination 

MSE ... mean squared error, the average squared difference between the estimated values and the actual value

RPD ... residual prediction deviation;  RPD is calculated as the ratio of the standard deviation of reference (training data set) soil property values to the RMSE of prediction (Viscarra Rossel, 2007)

**Three scenarios of model quality** 

Scenario A: moderate quality PLSR model with R2 ≥ 0.6, RPD ≥ 1.5

Scenario B: good quality PLSR model with R2 ≥ 0.7 and RPD ≥ 1.5

Scenario C: the best quality PLSR model from the given data set with R ≥ 0.8 and RPD ≥ 2.0 

In [ ]:
# test with up to selected maximun number of components
# more components require more calculations! 
max_comp = 20

# select your pre-processed input data (X, y) 
prep_X = X
prep_y = y_sqrt

# run the 'grid search' with cross-validation
r2s =  []
mses = []
rpds = []
xticks = np.arange(1, max_comp + 1)
for n_comp in xticks:
    y_cv, r2, mse, rpd = optimise_pls_cv(prep_X, prep_y, n_comp)
    r2s.append(r2)
    mses.append(mse)
    rpds.append(rpd)

In [ ]:
# Plot the calcualted metrics to see how the model performs for the range of components
def plot_metrics(vals, ylabel, objective):
    with plt.style.context('ggplot'):
        plt.plot(xticks, np.array(vals), '-o', color='blue', mfc='blue')
        if objective=='min':
            idx = np.argmin(vals)
        else:
            idx = np.argmax(vals)
        plt.plot(xticks[idx], np.array(vals)[idx], 'P', ms=10, mfc='red')

        plt.xlabel('Number of PLSR components')
        plt.xticks = xticks
        plt.ylabel(ylabel)
        plt.title('PLSR')

    plt.show()

In [ ]:
plot_metrics(mses, 'MSE', 'min')

In [ ]:
plot_metrics(rpds, 'RPD', 'max')

In [ ]:
plot_metrics(r2s, 'R2', 'max')

In [ ]:
# Notice that all the metrics confirm that X components is the best option.

In [ ]:
# place here your (model) selection
n_components = 11

In [ ]:
# caculate the predictions and quality metrics for the selected model
y_cv, r2, mse, rpd = optimise_pls_cv(prep_X, prep_y, n_components)

In [ ]:
print('Our model quality is: R2=%0.4f, MSE=%0.4f, RPD=%0.4f' %(r2, mse, rpd))

In [ ]:
# Is it reasonable result? 
# Compare it with existing literature publications! 

In [ ]:
# if you scaled the original SOC value, you should rescale back 
y_cv_ = y_cv ** 2 
prep_y_ = prep_y ** 2

In [ ]:
# plot the scatterplot with the seelcted regression model 
plt.figure(figsize=(6, 6))
with plt.style.context('ggplot'):
    plt.scatter(prep_y_, y_cv_, color='red', alpha=0.7)
    z = np.polyfit(prep_y_, y_cv_, 1)
    plt.plot(prep_y_, np.polyval(z, prep_y_), color='blue', 
             linewidth=0.5, linestyle='--', 
             label='Predicted regression line')
    plt.xlabel('Actual SOC (%)')
    plt.ylabel('Predicted SOC (%)')
    plt.legend()
    plt.plot()

### Tasks for students

* 1/ Split the input data set into train and test set to properly evlaute the model!
* 2/ Evluate what is the histogram asymatry influence on the model quality; 
* 3/ Test several differnt spectra pre-treatments; 
* 4/ Collect your own spectra of given soil sample with FieldSpec and predict its soil organic carbon concentration (%) using the above developed predictive model! 
* 5/ Optionally, replace the PLSR model with non-linear model, any; 